# Speech Emotion Detection Classifier

## 1. Import all the necessary libraries

In [1]:
# Core libraries
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

# Audio processing libraries
import librosa
import librosa.display
import soundfile as sf 

# Visualization libraries
import IPython.display as ipd
from IPython.display import Audio

# Machine learning libraries
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Deep Learning libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
import tensorflow as tf 


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## 2. Data Loading

In [2]:
Ravdess = "ravdess/audio_speech_actors_01-24/"
Crema = "crema/AudioWAV/"
Tess = "tess/TESS Toronto emotional speech set data/"
Savee = "savee/ALL/"

### 2.1 Data Exploration

#### 2.1.1 Function to explore datasets 

In [3]:
def explore_dataset(dataset_path):
    file_list = [f for f in os.listdir(dataset_path) if f.endswith('.wav')]
    print(f"Sample files in {dataset_path}:")
    print(file_list[:5])
    return file_list

ravdess_files = explore_dataset(Ravdess)
crema_files = explore_dataset(Crema)
tess_files = explore_dataset(Tess)
savee_files = explore_dataset(Savee)

Sample files in ravdess/audio_speech_actors_01-24/:
[]
Sample files in crema/AudioWAV/:
['1001_DFA_ANG_XX.wav', '1001_DFA_DIS_XX.wav', '1001_DFA_FEA_XX.wav', '1001_DFA_HAP_XX.wav', '1001_DFA_NEU_XX.wav']
Sample files in tess/TESS Toronto emotional speech set data/:
[]
Sample files in savee/ALL/:
['DC_a01.wav', 'DC_a02.wav', 'DC_a03.wav', 'DC_a04.wav', 'DC_a05.wav']


### 2.1.2 Map Integers to Emotions


#### 2.1.2.1 Ravdess Dataset

Update explore_dataset to Recursively List Files: 

We modify the function to list all files within the subfolders (e.g., for Actor_01 to Actor_24).

In [4]:
def explore_dataset(dataset_path):
    file_list = []
    # Loop through all subdirectories (Actor_01, Actor_02, etc.)
    for actor_folder in os.listdir(dataset_path):
        actor_folder_path = os.path.join(dataset_path, actor_folder)
        if os.path.isdir(actor_folder_path):  # Ensure it's a folder
            # Add all .wav files from each actor's folder to the list
            files = [f for f in os.listdir(actor_folder_path) if f.endswith('.wav')]
            file_list.extend(files)  # Add files to the overall list
    print(f"Sample files in {dataset_path}:")
    print(file_list[:5])  # Print the first 5 files
    return file_list


Since the emotion code is embedded in the filename but structured differently (e.g., something like Actor_01-03-01-01-01-01.wav), we extract the emotion code accordingly. 

If the emotion code is at a specific position in the filename, modify the extraction logic.

In [5]:
def map_emotion_ravdess(filename):
    parts = filename.split('-')
    emotion_code = int(parts[2])
    emotion_mapping = {
        1: 'neutral', 2: 'calm', 3: 'happy', 4: 'sad',
        5: 'angry', 6: 'fearful', 7: 'disgust', 8: 'surprised'
    }
    return emotion_mapping.get(emotion_code, "unknown")

ravdess_files = explore_dataset(Ravdess)
ravdess_mapped_emotions = [map_emotion_ravdess(f) for f in ravdess_files[:30]]
for emotion in ravdess_mapped_emotions:
    print(emotion)

Sample files in ravdess/audio_speech_actors_01-24/:
['03-01-01-01-01-01-01.wav', '03-01-01-01-01-02-01.wav', '03-01-01-01-02-01-01.wav', '03-01-01-01-02-02-01.wav', '03-01-02-01-01-01-01.wav']
neutral
neutral
neutral
neutral
calm
calm
calm
calm
calm
calm
calm
calm
happy
happy
happy
happy
happy
happy
happy
happy
sad
sad
sad
sad
sad
sad
sad
sad
angry
angry


#### 2.1.2.2 Crema Dataset

In [6]:
def explore_dataset(directory_path):
    print(f"Exploring directory: {directory_path}")
    files = os.listdir(directory_path)
    print(f"Files found: {files}")
    return files

crema_files = explore_dataset(Crema)


Exploring directory: crema/AudioWAV/
Files found: ['1001_DFA_ANG_XX.wav', '1001_DFA_DIS_XX.wav', '1001_DFA_FEA_XX.wav', '1001_DFA_HAP_XX.wav', '1001_DFA_NEU_XX.wav', '1001_DFA_SAD_XX.wav', '1001_IEO_ANG_HI.wav', '1001_IEO_ANG_LO.wav', '1001_IEO_ANG_MD.wav', '1001_IEO_DIS_HI.wav', '1001_IEO_DIS_LO.wav', '1001_IEO_DIS_MD.wav', '1001_IEO_FEA_HI.wav', '1001_IEO_FEA_LO.wav', '1001_IEO_FEA_MD.wav', '1001_IEO_HAP_HI.wav', '1001_IEO_HAP_LO.wav', '1001_IEO_HAP_MD.wav', '1001_IEO_NEU_XX.wav', '1001_IEO_SAD_HI.wav', '1001_IEO_SAD_LO.wav', '1001_IEO_SAD_MD.wav', '1001_IOM_ANG_XX.wav', '1001_IOM_DIS_XX.wav', '1001_IOM_FEA_XX.wav', '1001_IOM_HAP_XX.wav', '1001_IOM_NEU_XX.wav', '1001_IOM_SAD_XX.wav', '1001_ITH_ANG_XX.wav', '1001_ITH_DIS_XX.wav', '1001_ITH_FEA_XX.wav', '1001_ITH_HAP_XX.wav', '1001_ITH_NEU_XX.wav', '1001_ITH_SAD_XX.wav', '1001_ITS_ANG_XX.wav', '1001_ITS_DIS_XX.wav', '1001_ITS_FEA_XX.wav', '1001_ITS_HAP_XX.wav', '1001_ITS_NEU_XX.wav', '1001_ITS_SAD_XX.wav', '1001_IWL_ANG_XX.wav', '1001_

In [7]:
def map_emotion_crema(filename):
    parts = filename.split('_')
    emotion_code = parts[2]
    emotion_mapping = {
        'NEU': 'neutral', 'HAP': 'happy', 'SAD': 'sad', 
        'ANG': 'angry', 'DIS': 'disgust', 'FEA': 'fearful', 
        'SUR': 'surprised'
    }
    return emotion_mapping.get(emotion_code, "unknown")

crema_files = explore_dataset(Crema)
crema_mapped_emotions = [map_emotion_crema(f) for f in crema_files[:30]]
for emotion in crema_mapped_emotions:
    print(emotion)

Exploring directory: crema/AudioWAV/
Files found: ['1001_DFA_ANG_XX.wav', '1001_DFA_DIS_XX.wav', '1001_DFA_FEA_XX.wav', '1001_DFA_HAP_XX.wav', '1001_DFA_NEU_XX.wav', '1001_DFA_SAD_XX.wav', '1001_IEO_ANG_HI.wav', '1001_IEO_ANG_LO.wav', '1001_IEO_ANG_MD.wav', '1001_IEO_DIS_HI.wav', '1001_IEO_DIS_LO.wav', '1001_IEO_DIS_MD.wav', '1001_IEO_FEA_HI.wav', '1001_IEO_FEA_LO.wav', '1001_IEO_FEA_MD.wav', '1001_IEO_HAP_HI.wav', '1001_IEO_HAP_LO.wav', '1001_IEO_HAP_MD.wav', '1001_IEO_NEU_XX.wav', '1001_IEO_SAD_HI.wav', '1001_IEO_SAD_LO.wav', '1001_IEO_SAD_MD.wav', '1001_IOM_ANG_XX.wav', '1001_IOM_DIS_XX.wav', '1001_IOM_FEA_XX.wav', '1001_IOM_HAP_XX.wav', '1001_IOM_NEU_XX.wav', '1001_IOM_SAD_XX.wav', '1001_ITH_ANG_XX.wav', '1001_ITH_DIS_XX.wav', '1001_ITH_FEA_XX.wav', '1001_ITH_HAP_XX.wav', '1001_ITH_NEU_XX.wav', '1001_ITH_SAD_XX.wav', '1001_ITS_ANG_XX.wav', '1001_ITS_DIS_XX.wav', '1001_ITS_FEA_XX.wav', '1001_ITS_HAP_XX.wav', '1001_ITS_NEU_XX.wav', '1001_ITS_SAD_XX.wav', '1001_IWL_ANG_XX.wav', '1001_

#### 2.1.2.3 Tess Dataset

In [8]:
import os

def explore_dataset(directory):
    file_list = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".wav"):
                file_list.append(os.path.join(root, file))
                
    return file_list
tess_path = "tess/TESS Toronto emotional speech set data/"

print(tess_files[:10])
                

[]


In [9]:
def map_emotion_tess(filename):
    emotion_keywords = {
        "neutral": "neutral", "happy": "happy", "sad": "sad",
        "angry": "angry", "fear": "fearful", "disgust": "disgust",
        "ps": "surprised"
    }
    for key, emotion in emotion_keywords.items():
        if key in filename.lower():
            return emotion
    return "unknown"

tess_files = explore_dataset(tess_path)
tess_mapped_emotions = [map_emotion_tess(f) for f in tess_files[:30]]

print("\nTESS Emotions:")
print("\n".join(tess_mapped_emotions))


TESS Emotions:
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry
angry


#### 2.1.2.4 Savee Dataset

First we get a Function to explore the dataset and list all audio files


In [11]:
def explore_dataset(directory):
    file_list = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".wav"):
                file_list.append(file)
    return file_list 

savee_path = "savee/ALL/"
savee_files = explore_dataset(savee_path)

print(savee_files[:10])           

['DC_a01.wav', 'DC_a02.wav', 'DC_a03.wav', 'DC_a04.wav', 'DC_a05.wav', 'DC_a06.wav', 'DC_a07.wav', 'DC_a08.wav', 'DC_a09.wav', 'DC_a10.wav']


Let's map emotions

In [12]:
def map_emotion_savee(filename):
    emotion_mapping = {
        "n": "neutral", "h": "happy", "sa": "sad", "a": "angry",
        "f": "fearful", "d": "disgust", "su": "surprised"
    }
    # Extracting the first characters of the filename for emotion mapping
    code = filename[3:5] if filename[3:5] in emotion_mapping else filename[3]
    return emotion_mapping.get(code, "unknown")

savee_files = explore_dataset(savee_path)

savee_mapped_emotions = [(f, map_emotion_savee(f)) for f in savee_files]

print("SAVEE Dataset - Mapped Emotions:")
for file, emotion in savee_mapped_emotions[:30]:
    print(f"{file} → {emotion}")

SAVEE Dataset - Mapped Emotions:
DC_a01.wav → angry
DC_a02.wav → angry
DC_a03.wav → angry
DC_a04.wav → angry
DC_a05.wav → angry
DC_a06.wav → angry
DC_a07.wav → angry
DC_a08.wav → angry
DC_a09.wav → angry
DC_a10.wav → angry
DC_a11.wav → angry
DC_a12.wav → angry
DC_a13.wav → angry
DC_a14.wav → angry
DC_a15.wav → angry
DC_d01.wav → disgust
DC_d02.wav → disgust
DC_d03.wav → disgust
DC_d04.wav → disgust
DC_d05.wav → disgust
DC_d06.wav → disgust
DC_d07.wav → disgust
DC_d08.wav → disgust
DC_d09.wav → disgust
DC_d10.wav → disgust
DC_d11.wav → disgust
DC_d12.wav → disgust
DC_d13.wav → disgust
DC_d14.wav → disgust
DC_d15.wav → disgust
